<a href="https://colab.research.google.com/github/heatshockprotein60/doh-dwh-dashboards/blob/master/SARI_Line_List_Processing_25_GB_%F0%9F%94%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Initialize Notebook**

Install Specific Dependencies

In [ ]:
# !cat /proc/cpuinfo

In [ ]:
%%capture
!pip install pandas_access # for reading MS Access database
# !sudo apt install mdbtools
# !pip install meza
# !pip install --upgrade string-grouper
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
!pip install gspread google-auth gspread-formatting
# !pip install matplotlib seaborn

In [ ]:
# Import libraries
import gspread
from oauth2client.client import GoogleCredentials
from gspread_formatting import *
# from gspread_formatting import HorizontalAlignment
import numpy as np
import pandas as pd
from IPython.display import HTML
# from fuzzywuzzy import fuzz, process
# import matplotlib.pyplot as plt
# import seaborn as sns
import os
import csv
import shutil
import glob
import re
import time
from datetime import timedelta
from datetime import datetime as dt
import datetime
import pytz
from google.colab import auth
from google.cloud import bigquery
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.oauth2 import service_account
import pandas_gbq as gbq
# from tqdm import tqdm
from google.colab import drive

# Allow pandas to show all rows
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Authentication - click the link, authenticate, and paste the given code
from google.colab import auth
auth.authenticate_user()

# Get service account
!gsutil cp gs://doh-covid19-ops/linelist-service-account.json service_account.json

from google.colab import drive
drive.mount('/content/drive')

# BigQuery project
project_id = 'doh-covid-dwh'
dataset = 'sari_source'

# Declare service account credentials for performing BQ actions
scopes = [
    "https://www.googleapis.com/auth/bigquery",
    "https://www.googleapis.com/auth/drive",
    'https://www.googleapis.com/auth/spreadsheets',
]
!gsutil cp gs://doh-covid19-ops/linelist-service-account.json service_account.json
bq_credentials = service_account.Credentials.from_service_account_file("service_account.json", scopes=scopes)

# Define Gspread client
gspread_client = gspread.authorize(bq_credentials)

# BigQuery client - for reading and writing to database
client = bigquery.Client(
    project_id,
    credentials=bq_credentials
    )

Copying gs://doh-covid19-ops/linelist-service-account.json...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Copying gs://doh-covid19-ops/linelist-service-account.json...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      


## **Define Global Variables**

### Run Date Checks

### Specifying dates



In [ ]:
from datetime import datetime, timedelta
import pytz

# Set the target year
target_year = 2024

# Get the current date and time
current_date = datetime.now()

# Get the refresh date in 'Asia/Manila' timezone
refresh_date = datetime.now(pytz.timezone('Asia/Manila')).date()
refresh_date_str = refresh_date.strftime("%Y-%m-%d")
refresh_date_tablename = refresh_date_str.replace("-", "")
print("Run Date: {}".format(refresh_date_str))

# Calculate the previous run date (5 days before the refresh date)
previous_date = refresh_date - timedelta(days=5)
previous_date_str = previous_date.strftime("%Y-%m-%d")
previous_date_tablename = previous_date_str.replace("-", "")
print("Previous Run Date: {}".format(previous_date_str))

# Calculate the current week's Friday
days_to_add = (4 - current_date.weekday()) % 7
current_week_friday = current_date + timedelta(days=days_to_add)
current_week_friday_str = current_week_friday.strftime("%Y-%m-%d")
current_week_friday_line_list = current_week_friday_str.replace("-", "")
print("Current Friday Line List: {}".format(current_week_friday_line_list))

# Calculate the current and previous week numbers
current_week_number = current_date.isocalendar()[1]
previous_week_number = current_week_number - 1
previous_week_table_name = 'pidsr_source.sari_mw_{:02d}'.format(previous_week_number)
print("Previous week table name:", previous_week_table_name)

# Calculate last week's Saturday
days_to_subtract = (current_date.weekday() + 2) % 7  # Adding 1 to get Saturday of last week
last_week_saturday = current_date - timedelta(days=days_to_subtract)
last_week_saturday_str = last_week_saturday.strftime("%Y-%m-%d")
last_week_saturday_line_list = last_week_saturday_str.replace("-", "")
print("Last Week Saturday Line List: {}".format(last_week_saturday_line_list))

# Calculate the last two Sundays
days_to_sunday = (current_date.weekday() + 1) % 7  # Current week's Sunday
current_sunday = current_date - timedelta(days=days_to_sunday)
current_sunday_str = current_sunday.strftime("%Y-%m-%d")
current_sunday_line_list = current_sunday_str.replace("-", "")
print("Current Sunday Line List: {}".format(current_sunday_line_list))

last_sunday = current_sunday - timedelta(days=7)
last_sunday_str = last_sunday.strftime("%Y-%m-%d")
last_sunday_line_list = last_sunday_str.replace("-", "")
print("Last Sunday Line List: {}".format(last_sunday_line_list))

# Calculate the last two Fridays
last_friday = current_week_friday - timedelta(days=7)
last_friday_str = last_friday.strftime("%Y-%m-%d")
last_friday_line_list = last_friday_str.replace("-", "")
print("Last Friday Line List: {}".format(last_friday_line_list))

Run Date: 2024-07-18
Previous Run Date: 2024-07-13
Current Friday Line List: 20240719
Previous week table name: pidsr_source.sari_mw_28
Last Week Saturday Line List: 20240713
Current Sunday Line List: 20240714
Last Sunday Line List: 20240707
Last Friday Line List: 20240712


## **Processing ILI Data from TKC**

In [ ]:
tkc_upload = input("Are you able to upload latest TKC extract? (Y/N): ")

while tkc_upload != "Y" and tkc_upload != "N":
  tkc_upload = input("Are you able to upload latest TKC extract? (Y/N): ")
else:
  if tkc_upload == "Y":
    print("\nProceeding with the process.")
  else:
    raise Exception("\n\nPlease upload latest file at https://drive.google.com/drive/u/3/folders/1odpQi1CL_fg04Q-JCsTMPYKWk0qlWmyL before running the code below.")

Are you able to upload latest TKC extract? (Y/N): Y

Proceeding with the process.


### **Accessing latest file from TKC extract folder**

In [ ]:
# Folder ID of TKC Extracts folder
# link: https://drive.google.com/drive/u/3/folders/1odpQi1CL_fg04Q-JCsTMPYKWk0qlWmyL
folder_id = '1odpQi1CL_fg04Q-JCsTMPYKWk0qlWmyL'

# Authenticate and create PyDrive client
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List all files in the folder
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# Find the CSV file for current MW
tkc_file_name = "SARI MW" + str(previous_week_number) + ".csv"

# Search for the file in the specified folder
target_file = None
for file in file_list:
  if file['title'] == tkc_file_name:
    target_file = file
    break

if target_file:
  # Get the file ID
  target_file_id = target_file['id']

  # Download the file
  file_path = f"/content/{tkc_file_name}"
  file = drive.CreateFile({'id': target_file_id})
  file.GetContentFile(file_path)

  dtype_tkc = {'house_village_name': str,
               'apartment_unit_number': str,
               'apartment_floor_number': str,
               'permanent_address_city': str,
               'has_travel_abroad': str,
               'outcome': str
               }

  tkc_extract_df = pd.read_csv(file_path, dtype=dtype_tkc)
  print("\nSuccessfully read " + tkc_file_name + ".\n")
else:
  print("\nFile '{tkc_file_name}' not found or not yet uploaded in the TKC Extracts GDrive folder.")


Successfully read SARI MW28.csv.



### **Initial cleaning and uploading to BQ**

In [ ]:
# Initial cleaning of TKC extract before uploading to BQ

## change column names
# tkc_extract_df['admission_date'] = tkc_extract_df['addmission_date']

# ## fix format of date columns -- fix in BQ instead
# date_columns = ['created_at', 'birthdate', 'date_death',
#                 'admission_date', 'date_onset_of_illness',
#                 'date_collected', 'vaccination_date']

# for col in date_columns:
#   tkc_extract_df[col] = tkc_extract_df[col].strip()
#   tkc_extract_df[f"{col}_formatted"] = pd.to_datetime(tkc_extract_df[col], errors='coerce', format='mixed')
#   tkc_extract_df[f"{col}_formatted"] = tkc_extract_df[f"{col}_formatted"].dt.strftime('%Y-%m-%d')

## drop unnecessary variables
to_delete_vars = ['type', 'tkc_id', 'civil_status', 'telephone_number', 'cellphone_number', 'email_address',
                  'current_address_district', 'current_address_house_number', 'permanent_address_district', 'employer_name', 'work_address',
                  ]

tkc_extract_df = tkc_extract_df.drop(to_delete_vars, axis=1)

## convert NaN values in object columns to None
tkc_extract_df = tkc_extract_df.where(pd.notnull(tkc_extract_df), None)

## convert string values to UPPER case
# Convert 'object' columns to string type
object_columns = tkc_extract_df.select_dtypes(include=['object']).columns
for col in object_columns:
    tkc_extract_df[col] = tkc_extract_df[col].astype(str)

# Convert all values to uppercase
for col in object_columns:
    tkc_extract_df[col] = tkc_extract_df[col].str.upper()

# Convert back NONEs to None
tkc_extract_df = tkc_extract_df.applymap(lambda x: None if x == 'NONE' else x)

In [ ]:
# Upload TKC data to BigQuery

tablename = "sari_source.10_sari_tkc_" + refresh_date_tablename
tablename_latest = "sari_source.10_sari_tkc_latest"

for table in [tablename, tablename_latest]:
  tkc_extract_df.to_gbq(destination_table=f"{project_id}.{table}",
                        project_id=project_id,
                        if_exists="replace",
                        credentials=bq_credentials)
  print(f'Successfully materialized {table}')

### **Pre-process ILI Data from TKC**

In [ ]:
query = """CALL `{}.{}.sp_tkc_cleaning`()""".format(project_id, dataset)

job = client.query(query)
job.result()

## **Processing SARI Data from EDCS-IS**

In [ ]:
query_column_check = "SELECT * FROM `edcs_10_source.sari_view_lab_latest` LIMIT 1"

query_column_check = gbq.read_gbq(query_column_check, project_id = project_id, credentials = bq_credentials)
print("Row count: ", query_precleaned.shape[0])
df_21 = query_column_check

In [ ]:
columns = [
    "ACINETOBACTER CALCOACETICUS","ACINETOBACTER CALCOACETICUS, RSV","ACINETOBACTER LWOFFII","CHLAMYDIA PNEUMONIAE","ENTEROBACTER CLOACAE","HADV","HADV, HBOV",
    "HADV, HPIV-4","HADV, KLEBSIELLA PNEUMONIAE","HAEMOPHILUS INFLUENZAE","HAEMOPHILUS INFLUENZAE, HMPV","HBOV","HBOV, HADV","HBOV, HRV","HCOV","HCOV-229E","HCOV-229E, HMPV","HCOV-HKU1",
    "HCOV-NL63","HCOV-NL63, HADV","HCOV-NL63, HMPV","HCOV-OC43","HCOV-OC43, HADV","HCOV-OC43, HBOV","HCOV-OC43, HEV","HCOV-OC43, HPIV-1","HCOV-OC43, HPIV-4","HCOV-OC43, HRV",
    "HEV","HEV, HRV","HEV, HRV, RSV","HEV, RSV","HMPV","HMPV, HADV","HMPV, HBOV","HMPV, HPEV","HPEV","HPIV","HPIV-1","HPIV-1, HADV","HPIV-1, HBOV","HPIV-2","HPIV-2, HADV",
    "HPIV-2, HBOV","HPIV-3","HPIV-3, HADV","HPIV-3, HBOV","HPIV-3, HBOV, HADV","HPIV-3, HRV","HPIV-4","HPIV-4, HRV","HRV","HRV, EVM HADV","HRV, HBOV, HADV","HRV, HBOV, HEV",
    "HRV, HCOV-229E, HBOV","HRV, HCOV-NL63","HRV, HCOV-NL63, HEV","HRV, HCOV-OC43","HRV, HEV, HADV","HRV, HEV, HPEV","HRV, HMPV","HRV, HMPV, HADV","HRV, HMPV, HBOV",
    "HRV, HPEV","HRV, HPIV-1","HRV, HPIV-3","HRV, HPIV-3, HBOV","MYCOPLASMA PNEUMONIAE, HRV","HRV,HBOV, HEV","INFLUENZA A","INFLUENZA A (H1N1)","INFLUENZA A (H1N1), HRV",
    "INFLUENZA A (H3)","INFLUENZA A (H3), RSV","INFLUENZA A, INFLUENZA B","INFLUENZA A, RSV","INFLUENZA B","INFLUENZA B (VICTORIA-LINEAGE)","INFLUENZA B (VICTORIA-LINEAGE), RSV",
    "INFLUENZA B (VICTORIA-LINEAGE), SARS-COV-2","KLEBSIELLA PNEUMONIAE","MORAXELLA CATARRHALIS","MYCOPLASMA PNEUMONIAE","PSEUDOMONAS AERUGINOSA","RSV","RSV-A","RSV-B",
    "SARS-COV-2","SARS-COV-2, RSV","STAPHYLOCOCCUS AUREUS","STREPTOCOCCUS PNEUMONIAE","HEV, HPIV-3"
]

In [ ]:
# check if schema is as expected
missing_fields = [column for column in df_21.columns if column not in standard_column_list]
new_fields = [column for column in columns if column not in df_21.columns]
if len(missing_fields) != 0 or len(new_fields) != 0:
  raise Exception(f"There are missing/new fields in today's extract! Please check if this is expected!! \nMissing Fields: {len(missing_fields)}\nNew Fields: {len(new_fields)}")
print("There are no missing/new fields in today's extract! Please proceed!!")

### **Extract SARI Data from EDCS-IS**

In [ ]:
query = """CALL `{}.{}.sp_edcs_extraction`()""".format(project_id, dataset)

job = client.query(query)
job.result()

### **Pre-process SARI Data from EDCS-IS**

In [ ]:
query = """CALL `{}.{}.sp_edcs_cleaning`()""".format(project_id, dataset)

job = client.query(query)
job.result()

In [ ]:
query_precleaned = """SELECT * FROM `{}.{}.15_sari_edcs_latest`""".format(project_id, dataset)

query_precleaned = gbq.read_gbq(query_precleaned, project_id = project_id, credentials = bq_credentials)
print("Row count: ", query_precleaned.shape[0])
df_21 = query_precleaned

Downloading: 100%|██████████|
Row count:  8653


**Perform Checks On Organism**

In [ ]:
# Define list of organisms
raw_organism_class = [
    "ACINETOBACTER CALCOACETICUS","ACINETOBACTER CALCOACETICUS, RSV","ACINETOBACTER LWOFFII","CHLAMYDIA PNEUMONIAE","ENTEROBACTER CLOACAE","HADV","HADV, HBOV",
    "HADV, HPIV-4","HADV, KLEBSIELLA PNEUMONIAE","HAEMOPHILUS INFLUENZAE","HAEMOPHILUS INFLUENZAE, HMPV","HBOV","HBOV, HADV","HBOV, HRV","HCOV","HCOV-229E","HCOV-229E, HMPV","HCOV-HKU1",
    "HCOV-NL63","HCOV-NL63, HADV","HCOV-NL63, HMPV","HCOV-OC43","HCOV-OC43, HADV","HCOV-OC43, HBOV","HCOV-OC43, HEV","HCOV-OC43, HPIV-1","HCOV-OC43, HPIV-4","HCOV-OC43, HRV",
    "HEV","HEV, HRV","HEV, HRV, RSV","HEV, RSV","HMPV","HMPV, HADV","HMPV, HBOV","HMPV, HPEV","HPEV","HPIV","HPIV-1","HPIV-1, HADV","HPIV-1, HBOV","HPIV-2","HPIV-2, HADV",
    "HPIV-2, HBOV","HPIV-3","HPIV-3, HADV","HPIV-3, HBOV","HPIV-3, HBOV, HADV","HPIV-3, HRV","HPIV-4","HPIV-4, HRV","HRV","HRV, EVM HADV","HRV, HBOV, HADV","HRV, HBOV, HEV",
    "HRV, HCOV-229E, HBOV","HRV, HCOV-NL63","HRV, HCOV-NL63, HEV","HRV, HCOV-OC43","HRV, HEV, HADV","HRV, HEV, HPEV","HRV, HMPV","HRV, HMPV, HADV","HRV, HMPV, HBOV",
    "HRV, HPEV","HRV, HPIV-1","HRV, HPIV-3","HRV, HPIV-3, HBOV","MYCOPLASMA PNEUMONIAE, HRV","HRV,HBOV, HEV","INFLUENZA A","INFLUENZA A (H1N1)","INFLUENZA A (H1N1), HRV",
    "INFLUENZA A (H3)","INFLUENZA A (H3), RSV","INFLUENZA A, INFLUENZA B","INFLUENZA A, RSV","INFLUENZA B","INFLUENZA B (VICTORIA-LINEAGE)","INFLUENZA B (VICTORIA-LINEAGE), RSV",
    "INFLUENZA B (VICTORIA-LINEAGE), SARS-COV-2","KLEBSIELLA PNEUMONIAE","MORAXELLA CATARRHALIS","MYCOPLASMA PNEUMONIAE","PSEUDOMONAS AERUGINOSA","RSV","RSV-A","RSV-B",
    "SARS-COV-2","SARS-COV-2, RSV","STAPHYLOCOCCUS AUREUS","STREPTOCOCCUS PNEUMONIAE","HEV, HPIV-3"
]

df_21['organism'] = df_21['organism'].str.upper().str.strip()
df_21e = df_21[['organism']][~df_21['organism'].isin(raw_organism_class)]
df_21e = df_21e.dropna()

if df_21e.shape[0] > 0:
  display(HTML(df_21e.to_html(index=False)))
  print("There are unidentified organisms with {} rows listed above. Perform code revisions on organisms!".format(df_21e.shape[0]))
else:
  print(f"All checks passed")

All checks passed


In [ ]:
# Define list of laboratory results
raw_lab_result_class = [
      "EQUIVOCAL","NEGATIVE","NON POLIO ENTERO VIRUS","NONE","PENDING","POSITIVE","JAPANESE ENCEPHALITIS","N.MENINGITIDIS","INDETERMINATE","NOT PROCESSED","UNKNOWN"
]

df_21g = df_21[['lab_result']][~df_21['lab_result'].isin(raw_lab_result_class)]
df_21g = df_21g.dropna()

if df_21g.shape[0] > 0:
  display(HTML(df_21g.to_html(index=False)))
  print("There are unidentified laboratory results with {} rows listed above. Perform code revisions on laboratory results!".format(df_21g.shape[0]))
else:
  print(f"All checks passed")

All checks passed


## **Processing SARI Data from Transmittals**

In [ ]:
# query = """CALL `{}.{}.sp_trans_cleaning`()""".format(project_id, dataset)

# job = client.query(query)
# job.result()

## **Merging of TKC and EDCS-IS data**

In [ ]:
query = """CALL `{}.{}.sp_merging`("{}", "{}", "{}", "{}")""".format(project_id, dataset, last_sunday_str, last_week_saturday_str, last_sunday_str, last_week_saturday_str)

job = client.query(query)
job.result()

### **Perform Quality Checks**

In [ ]:
# query EDCS
query_edcs = """
    SELECT
      COUNT(1) count
    FROM `ili_source.15_ili_edcs_latest`
    WHERE last_modified_date >= "{}" AND proxy_onset_date BETWEEN "2023-01-01" AND "2024-12-31"
""".format(lmd_start_date_str)

query_edcs = gbq.read_gbq(query_edcs, project_id = project_id, credentials = bq_credentials)
df_edcs = query_edcs


# query TKC
query_tkc = """
    SELECT
      COUNT(1) count
    FROM `ili_source.15_ili_tkc_latest`
    WHERE entry_date >= "{}" AND proxy_onset_date BETWEEN "2023-01-01" AND "2024-12-31"
""".format(lmd_start_date_str)

query_tkc = gbq.read_gbq(query_tkc, project_id = project_id, credentials = bq_credentials)
df_tkc = query_tkc

# query merged
query_merged = """SELECT COUNT(1) count FROM `ili_source.20_ili_merged_latest`"""

query_merged = gbq.read_gbq(query_merged, project_id = project_id, credentials = bq_credentials)
df_merged = query_merged

# Display row numbers of individual tables
print("\nCurrent EDCS extract row count: ", df_edcs['count'].iloc[0])
print("Current TKC extract row count: ", df_tkc['count'].iloc[0])
print("Current merged linelist row count: ", df_merged['count'].iloc[0])

# Checks
if df_merged['count'].iloc[0] > df_edcs['count'].iloc[0] + df_tkc['count'].iloc[0]:
  raise Exception("\nCheck merging of EDCS and TKC, total row count for merged should be at most "+ str(df_edcs['count'].iloc[0] + df_tkc['count'].iloc[0]) + ". Current row count is " + str(df_merged['count'].iloc[0]) + ".")
else:
  print("\nRow counts are as expected!")

Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|

Current EDCS extract row count:  3229
Current TKC extract row count:  1135
Current merged linelist row count:  4227

Row counts are as expected!


## **Checking for Duplicates**

### **Setting up manual dedup sheet**

In [ ]:
# Define function to read worksheet
def get_worksheet(spreadsheet_id, sheet_name):
    try:
        workbook = gspread_client.open_by_key(spreadsheet_id)
    except Exception as e:
        print("\nCopy error!")
        print("Please check the spreadsheet: https://docs.google.com/spreadsheets/d/{}".format(spreadsheet_id))
        print("Please make sure that the service account 'data-drop-service-account@doh-covid-dwh.iam.gserviceaccount.com' has access to the above sheet.")
        raise e

    sheet = workbook.worksheet(sheet_name)
    return sheet

worksheet = get_worksheet("1GZrM54aUfcNbz6mtM9cyCa_BT_0WSnOL2pFcYt4fai8", "manual_dedup")

### **Check records to be auto-dropped**

In [ ]:
# Checking if there are weird cases to be auto-dropped

query_checks = """ SELECT * FROM `{}.{}.vw_auto_drop_dups` """.format(project_id, dataset)

df_auto_drop = gbq.read_gbq(query_checks, project_id = project_id, credentials = bq_credentials)
print("\nCheck if the following is correct. No is_new = ""N"" should be dropped.\n")
print(df_auto_drop)

Downloading: 100%|██████████|

Check if the following is correct. No is_new = N should be dropped.

  auto_drop_tag   caseid_tag is_new   count
0  for dropping    no caseid      Y     370
1  for dropping  with caseid      N       8
2        retain    no caseid      Y    3857
3        retain  with caseid      N  276560


### **Generating Records for Manual Dedup**

In [ ]:
query_dedup = """ SELECT * FROM `{}.{}.vw_generate_duplicates` """.format(project_id, dataset)

query_job = client.query(query_dedup)
query_dedup = pd.read_gbq(query_dedup, project_id)
results_query_dedup = query_job.result()

### **Updating Manual Dedup Sheet**

In [ ]:
## Checking if dedup sheet is ready for update

dedup_ready = input("Have you cleared the 25_sari_latest Google Sheet (Y/N): ")

while dedup_ready != "Y" and dedup_ready != "N":
  dedup_ready = input("Have you cleared the 25_sari_latest Google Sheet (Y/N): ")
else:
  if dedup_ready == "Y":
    print("\nProceeding with the updating of manual dedup sheet.")
  else:
    raise Exception("Please go to: https://docs.google.com/spreadsheets/d/1GZrM54aUfcNbz6mtM9cyCa_BT_0WSnOL2pFcYt4fai8/edit#gid=0. \nClear all the contents of manual_dedup sheet!! Run the next cell once done.")

Have you cleared the 25_sari_latest Google Sheet (Y/N): Y

Proceeding with the updating of manual dedup sheet.


In [ ]:
import pandas as pd
import json
import numpy as np

# Define date field columns
date_columns = ['birth_date', 'report_date', 'consult_date', 'onset_date', 'proxy_onset_date', 'travel_date', 'arrival_date', 'exposure_date',
                 'admitted_date', 'discharged_date', 'icu_date', 'chestxray_result_date', 'date_specimen_collected', 'date_specimen_received',
                  'date_lab_result', 'vax_date', 'died_date']

# Define float64 field column
float_columns = ['age']

# Convert date columns to datetime
for col in date_columns:
    query_dedup[col] = pd.to_datetime(query_dedup[col], errors='coerce')

# Convert float columns to numeric
for col in float_columns:
    query_dedup[col] = pd.to_numeric(query_dedup[col], errors='coerce')

# Handle out-of-range float values with a custom function
def handle_float_values(value):
    if pd.notna(value) and not np.isfinite(value):
        return None
    return value

# Apply the custom function to all float columns
query_dedup[float_columns] = query_dedup[float_columns].applymap(handle_float_values)

# Convert Timestamp objects to string format during the apply step
query_dedup[date_columns] = query_dedup[date_columns].apply(lambda x: x.dt.strftime('%Y-%m-%d') if x.name in date_columns else x)

# Replace nan values with null for the entire DataFrame
query_dedup.replace({np.nan: None}, inplace=True)

# Convert DataFrame to a list of dictionaries
data_list = query_dedup.to_dict(orient='records')

# # Convert None values to null in the list of dictionaries during JSON conversion
# json_string = json.dumps(data_list, default=str, indent=2, allow_nan=False)

# Convert the DataFrame to a list of lists
rows = query_dedup.values.tolist()

# Update the worksheet starting from cell A2
worksheet.update('A2', rows)

<ipython-input-14-8a4cbc936670>:46: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A2', rows)


{'spreadsheetId': '1GZrM54aUfcNbz6mtM9cyCa_BT_0WSnOL2pFcYt4fai8',
 'updatedRange': 'manual_dedup!A2:DS441',
 'updatedRows': 440,
 'updatedColumns': 112,
 'updatedCells': 33271}

### **Initial Duplicate Checker**

In [ ]:
query_checks = """ SELECT * FROM `{}.{}.vw_duplicate_checker_v1` """.format(project_id, dataset)

query_job = client.query(query_checks)
query_checks = pd.read_gbq(query_checks, project_id)
results_query_checks = query_job.result()

query_checks

,n_key,fname,case_id,birth_date,proxy_onset_date,lab_result,region


In [ ]:
# Checking if there are weird cases to be dropped

query_manual_dedup = """
    SELECT
      IF(CAST(to_delete AS STRING) = "1", "for dropping", "retain") to_delete,
      IF(case_id IS NOT NULL, "with caseid", "no caseid") caseid_tag,
      is_new,
      COUNT(1) count
    FROM `sari_source.25_sari_latest`
    GROUP BY 1, 2, 3
    ORDER BY 1, 2, 3
"""

df_manual_drop = gbq.read_gbq(query_manual_dedup, project_id = project_id, credentials = bq_credentials)
# print("Check if the following is correct. No is_new = ""N"" should be dropped.")
print(df_manual_drop)

Downloading: 100%|██████████|
      to_delete   caseid_tag  is_new  count
0  for dropping    no caseid    True    217
1  for dropping  with caseid   False      3
2        retain  with caseid   False    220


## **Generating Cleaned Line List**

### **Materialize final cleaned linelist** - Weekly date-shard and latest tables

In [ ]:
query = """CALL `{}.{}.sp_generate_clean_table`()""".format(project_id, dataset)

job = client.query(query)
job.result()

tablename = "sari_source.30_sari_" + current_week_friday_line_list
tablename_latest = "sari_source.30_sari_weekly_latest"

query_clean = """ SELECT * FROM `{}.{}.30_sari_weekly_latest"` WHERE proxy_onset_date <= "{}" """.format(project_id, dataset,last_week_saturday_str)

for table in [tablename, tablename_latest]:
  job_config = bigquery.QueryJobConfig(destination=f"{project_id}.{table}")
  job_config.write_disposition = "WRITE_TRUNCATE"
  job = client.query(query_clean, job_config=job_config)
  job.result()
  print(f'Successfully materialized: {table}')

### **Final Duplicate Checker**

In [ ]:
query_checks = """ SELECT * FROM `{}.{}.vw_duplicate_checker_v2` """.format(project_id, dataset)

query_job = client.query(query_checks)
query_checks = pd.read_gbq(query_checks, project_id)
results_query_checks = query_job.result()

query_checks

,n_key,fname,case_id,birth_date,proxy_onset_date,lab_result,region
0,LALMOHAN,LAL MOHAN,SARI-PHL-2023-0009531,1950-01-01,2023-09-29,UNKNOWN,03
1,LALMOHAN,MOHAN LAL,SARI-PHL-2023-0007596,1950-01-01,2023-09-29,NEGATIVE,03
2,CAS-INGLANGGOYANROEL,ROEL LANGGOYAN CAS-ING,SARI-PHL-2024-0007222,1990-11-21,2024-05-02,NEGATIVE,CAR
3,CAS-INGLANGGOYANROEL,LANGGOYAN ROEL CAS-ING,SARI-PHL-2024-0004815,1990-11-21,2024-05-02,None,CAR
4,ANGELAVILAJOEHANNASAN,JOEHANNA SAN ANGEL AVILA,SARI-PHL-2024-0006821,2023-11-15,2024-05-23,NEGATIVE,CAR
5,ANGELAVILAJOEHANNASAN,JOEHANNA SAN ANGEL AVILA,SARI-PHL-2024-0006146,2024-03-04,2024-05-28,PENDING,CAR
6,LAXAMANARONALDOSIENES,SIENES RONALDO LAXAMANA,SARI-PHL-2023-0006394,1974-07-12,2023-10-06,UNKNOWN,CAR
7,LAXAMANARONALDOSIENES,RONALDO LAXAMANA SIENES,SARI-PHL-2023-0006244,1974-07-12,2023-10-06,POSITIVE,CAR
8,BUCLAOJANINESUYAM,JANINE SUYAM BUCLAO,SARI-PHL-2023-0003200,2015-07-18,2023-08-16,NEGATIVE,CAR
9,BUCLAOJANINESUYAM,JANINE BUCLAO SUYAM,SARI-PHL-2023-0001259,2015-07-18,2023-08-16,NEGATIVE,CAR


### **Manual Dropping of Duplicates**

Input identified duplicates in this sheet using case_id: https://docs.google.com/spreadsheets/d/1ltx1zfpMP8yQ2lBkBGL3DaAmayjdGXdg2fbupIt_RVQ/edit?gid=0#gid=0

In [ ]:
# Drop duplicates if necessary, uncomment this code if there were duplicates found from the above query!
tablename_weekly = 'sari_source.30_sari_' + current_week_friday_line_list
tablename_weekly_latest = 'sari_source.30_sari_weekly_latest'

query_clean = """
  SELECT
    retained.case_id AS case_id,
    *EXCEPT(case_id)
  FROM `sari_source.30_sari_weekly_latest` retained
  LEFT JOIN `sari_source.31_sari_duplicate_dropping` dropped
    USING(case_id)
  WHERE dropped.case_id IS NULL
""".format(project_id, dataset)

for table in [tablename_weekly, tablename_weekly_latest]:
  job_config = bigquery.QueryJobConfig(destination=f"{project_id}.{table}")
  job_config.write_disposition = "WRITE_TRUNCATE"
  job = client.query(query_clean, job_config=job_config)
  job.result()
  print(f'Successfully materialized: {table}')

Successfully materialized: sari_source.30_sari_20240719
Successfully materialized: sari_source.30_sari_weekly_latest


### **Materialize remarks column**

In [ ]:
tablename_weekly = 'sari_source.30_sari_' + current_week_friday_line_list
tablename_weekly_latest = 'sari_source.30_sari_weekly_latest'

query_clean = """ SELECT * FROM `{}.{}.vw_generate_remarks_column` """.format(project_id, dataset)

for table in [tablename_weekly, tablename_weekly_latest]:
  job_config = bigquery.QueryJobConfig(destination=f"{project_id}.{table}")
  job_config.write_disposition = "WRITE_TRUNCATE"
  job = client.query(query_clean, job_config=job_config)
  job.result()
  print(f'Successfully materialized: {table}')

Successfully materialized: sari_source.30_sari_20240719
Successfully materialized: sari_source.30_sari_weekly_latest


### **Materialize final cleaned linelist** - Previous 5 years + latest date-shard

In [ ]:
query = """CALL `{}.{}.sp_generate_unioned_latest`()""".format(project_id, dataset)

job = client.query(query)
job.result()

Successfully materialized: sari_source.30_sari_latest


### **Perform Quality Checks**

In [ ]:
# query final weekly
query_final = """
    SELECT
      COUNT(1) count
    FROM `sari_source.30_sari_weekly_latest`
"""

query_final = gbq.read_gbq(query_final, project_id = project_id, credentials = bq_credentials)
df_final = query_final

# query previous week
query_prev = """
    SELECT
      COUNT(1) count
    FROM `sari_source.30_sari_{}`
""".format(last_friday_line_list)
# .format(previous_linelist_date_tablename)

query_prev = gbq.read_gbq(query_prev, project_id = project_id, credentials = bq_credentials)
df_prev = query_prev

# number of cases from 30_ili_weekly_latest
final_count = df_final['count'].iloc[0]

# number of cases from previous week
prev_count = df_prev['count'].iloc[0]

# number of cases from 20_ili_merged_latest
tkc_edcs_new_count = df_merged['count'].iloc[0]

# number of cases that were auto-dropped
to_drop = df_auto_drop[
    (df_auto_drop['auto_drop_tag'] == 'for dropping') &
    (df_auto_drop['caseid_tag'] == 'no caseid') &
    (df_auto_drop['is_new'] == 'Y')
    ]
to_drop_auto_count = to_drop['count'].sum()

# number of cases that were auto-dropped
to_drop_manual = df_manual_drop[df_manual_drop['to_delete'] == 'for dropping']
to_drop_manual_count = to_drop_manual['count'].sum()

if final_count != prev_count + tkc_edcs_new_count - to_drop_auto_count - to_drop_manual_count:
  raise Exception("\nCurrent row count is only at "+ str(final_count) +". \nFinal row count must be "+ str(prev_count + tkc_edcs_new_count - to_drop_auto_count - to_drop_manual_count) +". Review discrepancy!")
else:
  print("\nChecks passed. Current row count matches expected!")

Downloading: 100%|██████████|


GenericGBQException: Reason: 404 Not found: Table doh-covid-dwh:sari_source.30_sari_20240712 was not found in location US; reason: notFound, message: Not found: Table doh-covid-dwh:sari_source.30_sari_20240712 was not found in location US

Location: US
Job ID: 8b6ca06a-a61c-4bdd-ba73-f56c4bf9d773


## **Generate Dropped Line List**

In [ ]:
query = """CALL `{}.{}.sp_generate_dropped_cases`()""".format(project_id, dataset)

job = client.query(query)
job.result()

## **Internal Case Bulletin**

Total cases/New cases/Total confirmed/New confirmed/Total death/New deaths

In [ ]:
query = """
    SELECT
      COUNT(1) AS total_cases,
      COUNTIF(is_new = "Y") AS new_cases,
      COUNTIF(final_case_classification = "C") AS total_confirmed,
      COUNTIF(final_case_classification = "C" AND is_new = "Y") AS new_confirmed,

      COUNTIF(outcome = "D") AS total_deaths,
      COUNTIF(outcome = "D" AND is_new = "Y") AS new_deaths
    FROM `{}.{}.30_sari_weekly_latest`
""".format(project_id, dataset)

cases_count = gbq.read_gbq(query, project_id = project_id, credentials = bq_credentials)

Downloading: 100%|██████████|


In [ ]:
total_cases = '{:,}'.format(cases_count['total_cases'].iloc[0])
new_cases = '{:,}'.format(cases_count['new_cases'].iloc[0])
total_confirmed = '{:,}'.format(cases_count['total_confirmed'].iloc[0])
new_confirmed = '{:,}'.format(cases_count['new_confirmed'].iloc[0])
total_deaths = '{:,}'.format(cases_count['total_deaths'].iloc[0])
new_deaths = '{:,}'.format(cases_count['new_deaths'].iloc[0])

### Get the Top Regions of New Cases

In [ ]:
query = """
    SELECT
      *,
      row_number() over(order by _count DESC) AS row_num
    FROM (
      SELECT
        `doh-covid-dwh.edcs_20_trans.fx_cb_reg`(region) AS region,
        count(1) AS _count,
      FROM `{}.{}.30_sari_weekly_latest`
      WHERE is_new = "Y"
      GROUP BY 1 ORDER BY 2 DESC
      LIMIT 5
    )
""".format(project_id, dataset)

top_reg_bd = gbq.read_gbq(query, project_id = project_id, credentials = bq_credentials)

Downloading: 100%|██████████|


In [ ]:
top1_reg = top_reg_bd.loc[top_reg_bd['row_num'] == 1, 'region'].iloc[0]
top1_cnt = '{:,}'.format(top_reg_bd.loc[top_reg_bd['row_num'] == 1, '_count'].iloc[0])
top2_reg = top_reg_bd.loc[top_reg_bd['row_num'] == 2, 'region'].iloc[0]
top2_cnt = '{:,}'.format(top_reg_bd.loc[top_reg_bd['row_num'] == 2, '_count'].iloc[0])
top3_reg = top_reg_bd.loc[top_reg_bd['row_num'] == 3, 'region'].iloc[0]
top3_cnt = '{:,}'.format(top_reg_bd.loc[top_reg_bd['row_num'] == 3, '_count'].iloc[0])
top4_reg = top_reg_bd.loc[top_reg_bd['row_num'] == 4, 'region'].iloc[0]
top4_cnt = '{:,}'.format(top_reg_bd.loc[top_reg_bd['row_num'] == 4, '_count'].iloc[0])
top5_reg = top_reg_bd.loc[top_reg_bd['row_num'] == 5, 'region'].iloc[0]
top5_cnt = '{:,}'.format(top_reg_bd.loc[top_reg_bd['row_num'] == 5, '_count'].iloc[0])

### Get the previous cases with updates

In [ ]:
query = """
    SELECT
      COUNTIF(prev_cc = "S" AND latest_cc = "C") AS s_to_c,
    FROM (
      SELECT
        a.final_case_classification AS latest_cc,
        b.final_case_classification AS prev_cc,
        IF(a.final_case_classification != b.final_case_classification, "diff", "same") AS if_same
      FROM `{}.{}.30_sari_weekly_latest` a
      LEFT JOIN `{}.{}.30_sari_{}` b
      ON a.case_id = b.case_id
    )
""".format(project_id, dataset, project_id, dataset, last_friday_line_list)

w_updates = gbq.read_gbq(query, project_id = project_id, credentials = bq_credentials)

Downloading: 100%|██████████|


In [ ]:
s_to_c = '{:,}'.format(w_updates['s_to_c'].iloc[0])

### Printout: Case Bulletin

In [ ]:
case_bulletin = """
[Internal Bulletin]
SARI Case Data as of Morbidity Week {0}

Total Cases: {1} [{2} cases added]

Top Regions of New Cases:
1. {3} - {4}
2. {5} - {6}
3. {7} - {8}
4. {9} - {10}
5. {11} - {12}

Total Confirmed Cases: {13}
   Newly added: {14}
Previous cases with update:
   Suspect to Confirmed: {15}

Total Deaths: {16} [{17} cases added]
""".format(previous_week_number,
           total_cases, new_cases,
           top1_reg, top1_cnt,
           top2_reg, top2_cnt,
           top3_reg, top3_cnt,
           top4_reg, top4_cnt,
           top5_reg, top5_cnt,
           total_confirmed, new_confirmed,
           s_to_c,
           total_deaths, new_deaths)

print(case_bulletin)


[Internal Bulletin]
SARI Case Data as of Morbidity Week 28

Total Cases: 14,607 [2,078 cases added]

Top Regions of New Cases:
1. Region IV-A - 617
2. Region XI - 410
3. CAR - 227
4. NCR - 177
5. Region IV-B - 164

Total Confirmed Cases: 1,129
   Newly added: 1
Previous cases with update:
   Suspect to Confirmed: 0

Total Deaths: 239 [10 cases added]



## **For Line List Sharing**

In [ ]:
region = ["01", "02", "03", "4A", "4B", "05", "06", "07", "08", "09", "10", "11", "12", "BARMM", "CAR", "CARAGA", "NCR"]

### **For Cleaned Line List**

In [ ]:
def save_regional_linelist(region):
  query_linelist = """
  SELECT
    *
  FROM `sari_source.30_sari_weekly_latest`
  WHERE region = "{}"
    AND EXTRACT(YEAR FROM proxy_onset_date) = 2023
    AND EXTRACT(YEAR FROM proxy_onset_date) = 2024
  ORDER BY case_id
  """.format(region)
  query_linelist = gbq.read_gbq(query_linelist, project_id=project_id, credentials=bq_credentials)

  # temp_name = "CLEANED_" + region + "_AMES_MW{}.xlsx".format(previous_week_number)
  temp_name = "CLEANED_" + region + "_EDCS_2024.xlsx"

  query_linelist.to_excel("drive/MyDrive/EDCS Linelist Sharing/REGION {}/SARI/".format(region) + temp_name, index=False)

  # del linelist
  print('Uploaded', temp_name)

def check_excel(region):
  # filename = "CLEANED_{}_AMES_MW{}.xlsx".format(region, previous_week_number)
  filename = "CLEANED_{}_EDCS_2024.xlsx".format(region)
  if filename not in os.listdir("drive/MyDrive/EDCS Linelist Sharing/REGION _{}/".format(region)):

    raise Exception("{} is missing! Please recheck your code!".format(filename))

In [ ]:
#save each region file as separate excels
region_list = ["01", "02", "03", "4A", "4B", "05", "06", "07", "08", "09", "10", "11", "12", "BARMM", "CAR", "CARAGA", "NCR"]
for region in region_list:
  save_regional_linelist(region)

Downloading: |          |


OSError: Cannot save file into a non-existent directory: 'drive/MyDrive/EDCS Linelist Sharing/REGION 02/ILI'

### **For Verification**

In [ ]:
def save_regional_linelist(region):
  query_linelist = """
  SELECT
    *
  FROM `sari_source.30_sari_weekly_latest`
  WHERE region = "{}"
    AND remarks IS NOT NULL
    AND EXTRACT(YEAR FROM proxy_onset_date) = 2023
    AND EXTRACT(YEAR FROM proxy_onset_date) = 2024
  ORDER BY case_id
  """.format(region)
  query_linelist = gbq.read_gbq(query_linelist, project_id=project_id, credentials=bq_credentials)

  # temp_name = "FOR_VERIFICATION_" + region + "_AMES_MW{}.xlsx".format(previous_week_number)
  temp_name = "FOR_VERIFICATION_" + region + "_EDCS_2024.xlsx".format(previous_week_number)

  query_linelist.to_excel("drive/MyDrive/EDCS Linelist Sharing/REGION {}/SARI/".format(region) + temp_name, index=False)

  # del linelist
  print('Uploaded', temp_name)

def check_excel(region):
  # filename = "FOR_VERIFICATION_{}_AMES_MW{}.xlsx".format(region, previous_week_number)
  filename = "FOR_VERIFICATION_{}_EDCS_2024.xlsx".format(region)
  if filename not in os.listdir("drive/MyDrive/EDCS Linelist Sharing/REGION _{}/".format(region)):

    raise Exception("{} is missing! Please recheck your code!".format(filename))

In [ ]:
#save each region file as separate excels
region_list = ["01", "02", "03", "4A", "4B", "05", "06", "07", "08", "09", "10", "11", "12", "BARMM", "CAR", "CARAGA", "NCR"]
for region in region_list:
  save_regional_linelist(region)

GenericGBQException: Reason: 400 Unrecognized name: remarks at [6:9]; reason: invalidQuery, location: query, message: Unrecognized name: remarks at [6:9]

Location: US
Job ID: 52d8b026-745d-4c3b-a521-4b7bc575ec32


## **END**

In [ ]:
from IPython.display import Audio
sound_file = 'https://www.soundjay.com/misc/small-bell-ring-01a.mp3'
Audio(sound_file, autoplay=True)

In [ ]:
raise Exception("Process ends here.")

Exception: Process ends here.